In [34]:
import numpy as np
import astra
from scipy.optimize import newton, minimize
import scipy.sparse as sp
import torch

import pylops

In [38]:
np.int = np.int32

nx, ny = 128, 128  # Image size
num_proj = 180  # Number of projections
angles = np.linspace(0, np.pi, num_proj, endpoint=False)  # Projection angles

geometry = astra.create_proj_geom('parallel', 1.0, 180, num_proj, angles)
vol_geom = astra.create_vol_geom(nx, ny)
proj_id = astra.create_projector('line', geometry, vol_geom)
matrix_id = astra.projector.matrix(proj_id)


H = astra.matrix.get(matrix_id)
H = sp.csr_matrix(H)

H = torch.sparse_csr_tensor(
    torch.tensor(H.indptr, dtype=torch.int32),
    torch.tensor(H.indices, dtype=torch.int32),
    torch.tensor(H.data, dtype=torch.float32),
    size=H.shape
)

In [18]:
H

tensor(crow_indices=tensor([    0,     1,     4,    10,    19,    31,    46,
                               64,    84,   108,   135,   164,   196,   232,
                              270,   311,   355,   402,   452,   505,   560,
                              619,   681,   745,   812,   883,   956,  1032,
                             1111,  1193,  1278,  1366,  1456,  1550,  1647,
                             1746,  1848,  1954,  2062,  2173,  2287,  2404,
                             2524,  2647,  2772,  2901,  3033,  3167,  3304,
                             3445,  3588,  3734,  3883,  4036,  4191,  4349,
                             4510,  4673,  4840,  5010,  5182,  5357,  5536,
                             5717,  5901,  6088,  6278,  6471,  6667,  6865,
                             7067,  7272,  7479,  7689,  7903,  8119,  8338,
                             8560,  8784,  9008,  9231,  9454,  9678,  9902,
                            10125, 10348, 10572, 10796, 11019, 11242, 11466,

In [17]:
import matplotlib.pyplot as plt

#plt.imshow(H.toarray(), cmap='gray', aspect='auto', vmin = 0, vmax = 0.03)
plt.colorbar()
plt.title("Visualization of Projection Matrix H")
plt.show()

TypeError: can't convert SparseCsr layout tensor to numpy. Use Tensor.dense() first.

In [19]:
def objective(x):
    X = torch.diag(x)
    FIM = H @ X @ H.T
    objective = torch.linalg.det(FIM)
    log_padded = torch.where(objective > 1e-8, np.log(objective), 0)
    return - log_padded

In [32]:
def derivative_obj(x, H):
    H = H.to_sparse()
    X = torch.diag(x)
    HXHT = torch.sparse.mm(torch.sparse.mm(H,X),H.T)
    
    def solve_HXHT(b):
        y = torch.linalg.solve(HXHT, b)  # Use torch's linear solver
        return y

    HXHT_inv_H = torch.stack([solve_HXHT(H[:, i]) for i in range(H.shape[1])], dim=1)
    
    # Compute diagonal elements efficiently
    C_diag = torch.einsum("ij,ji->i", H.T, HXHT_inv_H)
    
    return -C_diag

def deriv_geom(x):
    return -1 / x

def newton_equation(theta, c, eq_constraint):
    return torch.sum(1 / (c + theta)) - eq_constraint

def newton_derivative(theta, c):
    return -torch.sum(1 / (c + theta) ** 2)

In [33]:
x = torch.ones(H.shape[1]) / H.shape[1]
c = derivative_obj(x, H) - deriv_geom(x)
theta = 100

theta_sol = newton(newton_equation, theta, fprime=newton_derivative, args=(c,))

NotImplementedError: Could not run 'aten::as_strided' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::as_strided' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapsh